In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
from matplotlib.pyplot import rc_context
import scanpy.external as sce
import seaborn as sns
from scipy.stats import ranksums
from statannot import add_stat_annotation
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")

In [ ]:
adata = sc.read_h5ad(r'E:\DataStore\adata_total.h5ad')
adata = adata[~adata.obs['celltype'].isin(['Epithelial'])]
sc.pp.filter_genes(adata, min_cells=10)

In [ ]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
adata.raw = adata

In [ ]:
sc.pp.highly_variable_genes(adata, batch_key='PID', n_top_genes=3000)
adata = adata[:, adata.var.highly_variable]

In [ ]:
sc.pp.scale(adata)
sc.tl.pca(adata)
sce.pp.harmony_integrate(adata, 'PID', max_iter_harmony=50)
adata.obsm['X_pca'] = np.array(adata.obsm['X_pca_harmony'])
sc.pp.neighbors(adata)
sc.tl.leiden(adata, resolution=1.5)
sc.tl.umap(adata)

In [ ]:
sc.pl.matrixplot(adata, var_names={'Treg':['CTLA4', 'FOXP3','IL2RA', 'IL1R2','RTKN2','LAIR2',],
                              'CD4T':['IL7R','CD40LG'],
                              'CD8T':[ 'CD8A', 'CD8B'],
                              'Cycling':[ 'RRM2','HMGN2'],
                              'Neutrophil':['CSF3R','FCGR3B'],
                              'Myeloid':['C1QB', 'LYZ', 'AIF1', 'RNASE1'],
                              'Endothelial':[ 'PECAM1',  'TM4SF1',],
                              'NK':['GNLY', 'KLRD1', 'KLRF1', 'GZMB'],
                              'B':['MS4A1', 'BANK1', 'CD79A', 'TNFRSF13C', 'BCL11A','PAX5'],
                              #'Epithelial':['KRT18', 'KRT19', 'EPCAM', 'SOX4'],
                              'DC3':['LAMP3', 'CCR7', 'CCL19', 'CCL22'],
                              'pDC':['TCF4', 'TCL1A'],
                              'Plasmacyte':['MZB1', 'IGLL5'],
                              'Fibroblast':[ 'COL1A1', 'LUM'],
                              'Mast':['TPSAB1','KIT']}, 
             groupby='leiden', standard_scale='var', cmap='RdBu_r')
#Refine the annotation based on markers.

In [ ]:
sc.set_figure_params(scanpy=True, fontsize=14)
with rc_context({'figure.figsize': (7, 6)}):
    sc.pl.umap(adata, color=['celltype'],s=1.2, save='total_immune_cell.svg')

In [ ]:
sc.set_figure_params(scanpy=True, fontsize=14, figsize=(6, 4))
sc.pl.umap(adata, color=['type'], 
          save='annot_type_PR.svg',
          palette={
              'PT':'#068eb7',
              'RT':'#a80c11',
          })